# Subject Conversion data work with `pandas`

This is a notebook to work with our subject reconciliation data and start to analyze it a bit. Maybe clean it in certain ways, too.


In [1]:
import pandas as pd

## Starting off the csv way
(Hey, it's what I know best! We can try to replicate in Excel next)  
After we get pandas set up, let's first see if the data is ready:

In [2]:
!head subject_conversion.csv

Status,old_ARK,old_predicate,new_predicate,predicate match,new_role,old_label,updated_label,current DAMS_Label is correct,Action,ARK action ,Label action ,Note,clustering id,FAST_Label_Match_Updated_DAMS_Label,FAST_Label,FAST_URI,LC_Label_Match_Updated_DAMS_Label,LoC_Label,LoC_URI,AAT_Label_Match,AAT_Label,AAT_URI,VIAF_Label_Match,VIAF_Label,VIAF_URI,MARC_tag,Sample_Object_ARK,Object_Counts,Parts,Part_order,DAMS_ARK_Part_order,sort,source,simple or complex,jj,wikipedia_link,ll,mm
split from complex no match,http://library.ucsd.edu/ark:/20775/bb0001382f,dams:complexSubject,dams:geographic,False,,"Rancho San Regis (Baja California, Mexico)","Rancho San Regis (Baja California, Mexico)",,local,,,,,,,,,,,,,,,,,,http://library.ucsd.edu/dc/object/bb98306314,2,2,1,http://library.ucsd.edu/ark:/20775/bb0001382f-1,2,split from complex,complex,,,,
split from complex matched FAST,http://library.ucsd.edu/ark:/20775/bb0001382f,dams:complexSubject,dams:topic,False,,Pictorial works,Pictorial works,True

Now let's get `pandas` to get this as a dataframe and show us a snippet. We'll use the magic `%%time` which will tell us how quickly this large-ish (>3MB) csv will become a dataframe

In [10]:
# Specify no low memory check, since that was causing errors
%%time

df = pd.read_csv('subject_conversion.csv', low_memory=False)

CPU times: user 264 ms, sys: 40 ms, total: 304 ms
Wall time: 328 ms


We can get some general info about the dataframe with the `info` method

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18749 entries, 0 to 18748
Data columns (total 39 columns):
Status                                 16939 non-null object
old_ARK                                16939 non-null object
old_predicate                          16939 non-null object
new_predicate                          16938 non-null object
predicate match                        16938 non-null object
new_role                               945 non-null object
old_label                              16938 non-null object
updated_label                          16938 non-null object
current DAMS_Label is correct          16938 non-null object
Action                                 14204 non-null object
ARK action                             549 non-null object
Label action                           418 non-null object
Note                                   49 non-null object
clustering id                          13098 non-null object
FAST_Label_Match_Updated_DAMS_Label    10829 n

We can get a sense of the row/column breakdown by using the `shape` method

In [5]:
df.shape

(18749, 39)

The number 18,749 is a bit misleading, and must be an instance of blank rows being counted (there's many things that can happen with even the best Excel->csv conversions). From the `.info` method above, we see there's really 16,938 non-null rows

Now let's look at a little snippet at the beginning of the data

In [6]:
df.head()

,Status,old_ARK,old_predicate,new_predicate,predicate match,new_role,old_label,updated_label,current DAMS_Label is correct,Action,...,Parts,Part_order,DAMS_ARK_Part_order,sort,source,simple or complex,jj,wikipedia_link,ll,mm
0,split from complex no match,http://library.ucsd.edu/ark:/20775/bb0001382f,dams:complexSubject,dams:geographic,False,NaN,"Rancho San Regis (Baja California, Mexico)","Rancho San Regis (Baja California, Mexico)",NaN,local,...,2.0,1.0,http://library.ucsd.edu/ark:/20775/bb0001382f-1,2.0,split from complex,complex,NaN,NaN,NaN,NaN
1,split from complex matched FAST,http://library.ucsd.edu/ark:/20775/bb0001382f,dams:complexSubject,dams:topic,False,NaN,Pictorial works,Pictorial works,True,FAST,...,NaN,2.0,http://library.ucsd.edu/ark:/20775/bb0001382f-2,3.0,split from complex,complex,NaN,NaN,NaN,NaN
2,original no match,http://library.ucsd.edu/ark:/20775/bb0001383z,dams:corporateName,dams:corporateName,True,NaN,"University Communications, University of Calif...","University of California, San Diego. Universit...",False,VIAF,...,1.0,0.0,http://library.ucsd.edu/ark:/20775/bb0001383z,5.0,original,simple,NaN,NaN,NaN,NaN
3,split from complex no match,http://library.ucsd.edu/ark:/20775/bb0001384g,dams:complexSubject,dams:geographic,False,NaN,"Pozo Alemán (Baja California, Mexico)","Pozo Alemán (Baja California, Mexico)",True,local,...,2.0,1.0,http://library.ucsd.edu/ark:/20775/bb0001384g-1,8.0,split from complex,complex,NaN,NaN,NaN,NaN
4,split from complex matched FAST,http://library.ucsd.edu/ark:/20775/bb0001384g,dams:complexSubject,dams:topic,False,NaN,Pictorial works,Pictorial works,True,FAST,...,NaN,2.0,http://library.ucsd.edu/ark:/20775/bb0001384g-2,9.0,split from complex,complex,NaN,NaN,NaN,NaN


In [17]:
df['updated_label'].head()

0           Rancho San Regis (Baja California, Mexico)
1                                      Pictorial works
2    University of California, San Diego. Universit...
3                Pozo Alemán (Baja California, Mexico)
4                                      Pictorial works
Name: updated_label, dtype: object

In [18]:
df_deduped = df.drop_duplicates(subset='updated_label', keep='first', inplace=False)

In [30]:
# Original df.info (number of non-null rows) = 16,938
df_deduped.shape

(10018, 39)

In [28]:
df_deduped[55:65]

,Status,old_ARK,old_predicate,new_predicate,predicate match,new_role,old_label,updated_label,current DAMS_Label is correct,Action,...,Parts,Part_order,DAMS_ARK_Part_order,sort,source,simple or complex,jj,wikipedia_link,ll,mm
60,original no match,http://library.ucsd.edu/ark:/20775/bb0035510q,dams:geographic,dams:geographic,True,NaN,"Valle Trinidad (Baja California, Mexico)","Valle Trinidad (Baja California, Mexico)",True,local,...,1.0,0.0,http://library.ucsd.edu/ark:/20775/bb0035510q,118.0,original,simple,NaN,NaN,NaN,NaN
61,split from complex no match,http://library.ucsd.edu/ark:/20775/bb00355117,dams:complexSubject,dams:corporateName,False,NaN,"Misión de Guadalupe (Baja California, Mexico)",Misión Nuestra Señora de Guadalupe del Norte...,False,VIAF,...,2.0,1.0,http://library.ucsd.edu/ark:/20775/bb00355117-1,121.0,split from complex,complex,NaN,NaN,NaN,NaN
63,split from complex matched FAST,http://library.ucsd.edu/ark:/20775/bb0035512r,dams:complexSubject,dams:topic,False,NaN,Refugees,Refugees,True,FAST,...,NaN,1.0,http://library.ucsd.edu/ark:/20775/bb0035512r-1,125.0,split from complex,complex,NaN,NaN,NaN,NaN
64,split from complex matched FAST,http://library.ucsd.edu/ark:/20775/bb0035512r,dams:complexSubject,dams:geographic,False,NaN,Spain,Spain,True,LoC,...,NaN,2.0,http://library.ucsd.edu/ark:/20775/bb0035512r-2,126.0,split from complex,complex,NaN,NaN,NaN,NaN
65,split from complex matched FAST,http://library.ucsd.edu/ark:/20775/bb00355138,dams:complexSubject,dams:topic,False,NaN,Amnesty,Amnesty,True,FAST,...,NaN,1.0,http://library.ucsd.edu/ark:/20775/bb00355138-1,129.0,split from complex,complex,NaN,NaN,NaN,NaN
68,split from complex matched FAST,http://library.ucsd.edu/ark:/20775/bb0035514s,dams:complexSubject,dams:personalName,False,subject,"Franco, Francisco, 1892-1975","Franco, Francisco, 1892-1975",True,VIAF,...,2.0,1.0,http://library.ucsd.edu/ark:/20775/bb0035514s-1,134.0,split from complex,complex,NaN,NaN,NaN,NaN
69,split from complex matched FAST,http://library.ucsd.edu/ark:/20775/bb0035514s,dams:complexSubject,dams:topic,False,NaN,Caricatures and cartoons,Caricatures and cartoons,True,FAST,...,NaN,2.0,http://library.ucsd.edu/ark:/20775/bb0035514s-2,135.0,split from complex,complex,NaN,NaN,NaN,NaN
70,split from complex matched FAST,http://library.ucsd.edu/ark:/20775/bb00355159,dams:complexSubject,dams:topic,False,NaN,Ballot,Ballot,True,FAST,...,2.0,1.0,http://library.ucsd.edu/ark:/20775/bb00355159-1,138.0,split from complex,complex,NaN,NaN,NaN,NaN
71,split from complex matched FAST,http://library.ucsd.edu/ark:/20775/bb00355159,dams:complexSubject,dams:geographic,False,NaN,Costa Rica,Costa Rica,True,LoC,...,NaN,2.0,http://library.ucsd.edu/ark:/20775/bb00355159-2,139.0,split from complex,complex,NaN,NaN,NaN,NaN
72,split from complex matched FAST,http://library.ucsd.edu/ark:/20775/bb0035519c,dams:complexSubject,dams:topic,False,NaN,Missions,Missions,True,FAST,...,NaN,1.0,http://library.ucsd.edu/ark:/20775/bb0035519c-1,144.0,split from complex,complex,NaN,NaN,NaN,NaN


# Now it's time for Excel
We'll do basically what we did above, but read the original Excel file and attempt no conversions

In [25]:
%%time
dfe = pd.read_excel("combined_subject_conversion_file.xlsx", sheetname='ALL EM')

CPU times: user 12.1 s, sys: 132 ms, total: 12.3 s
Wall time: 12.6 s


### First goal: dropping some rows
Filter out / drop all complex objects, then proceed to dedupe as above.  
But first, let's do the usual data info check

In [29]:
dfe.shape

(16939, 38)

In [27]:
dfe['updated_label'].head

<bound method NDFrame.head of 0               Rancho San Regis (Baja California, Mexico)
1                                          Pictorial works
2        University of California, San Diego. Universit...
3                    Pozo Alemán (Baja California, Mexico)
4                                          Pictorial works
5                        Chimbu (Papua New Guinean people)
6                                                   Poetry
7                                           Projection art
8                                      Munk, Judith Horton
9                                 Dill, Robert F. (1927- )
10                                                   Drama
11                               Revelle, Ellen, 1910-2009
12                                                    Lava
13                                      Lockwood, Karen F.
14                                Hills, Scott Jean, 1955-
15          Kofoid, Charles A. (Charles Atwood), 1865-1947
16                        